In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.linear_model import RidgeClassifier

clf = RidgeClassifier()
clf.fit(X, Y)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model"  # "https://sklearn2sql.herokuapp.com/"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:2000])

WITH linear_model_cte AS 
(SELECT "ADS"."KEY" AS "KEY", 0.12705283360921124 * "ADS"."Feature_0" + 0.4695089303373806 * "ADS"."Feature_1" + -0.4427640409866786 * "ADS"."Feature_2" + -0.130707226701817 * "ADS"."Feature_3" + -0.688748826512 AS "Score_0", -0.030416160173984446 * "ADS"."Feature_0" + -0.8686439569055142 * "ADS"."Feature_1" + 0.36914586521603815 * "ADS"."Feature_2" + -0.8145271451057391 * "ADS"."Feature_3" + 2.08608735288 AS "Score_1", -0.09663667343522389 * "ADS"."Feature_0" + 0.39913502656813316 * "ADS"."Feature_1" + 0.07361817577063601 * "ADS"."Feature_2" + 0.9452343718075641 * "ADS"."Feature_3" + -2.39733852637 AS "Score_2" 
FROM "INPUT_DATA" AS "ADS"), 
orig_cte AS 
(SELECT linear_model_cte."KEY" AS "KEY", linear_model_cte."Score_0" AS "Score_0", linear_model_cte."Score_1" AS "Score_1", linear_model_cte."Score_2" AS "Score_2", CAST(NULL AS FLOAT(53)) AS "Proba_0", CAST(NULL AS FLOAT(53)) AS "Proba_1", CAST(NULL AS FLOAT(53)) AS "Proba_2", CAST(NULL AS FLOAT(53)) AS "LogP

## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);


In [9]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,-1.209011,-0.594751,0.803762,None,None,None,None,None,None,2
74,74,-0.587840,-0.099202,-0.312959,None,None,None,None,None,None,1
9,9,0.712071,-0.283482,-1.428589,None,None,None,None,None,None,0
88,88,-0.553978,-0.235562,-0.210460,None,None,None,None,None,None,2
25,25,0.620478,-0.244197,-1.376281,None,None,None,None,None,None,0
5,5,1.023440,-1.164134,-0.859305,None,None,None,None,None,None,0
48,48,1.031527,-0.898288,-1.133239,None,None,None,None,None,None,0
117,117,-1.180383,-0.767647,0.948030,None,None,None,None,None,None,2
83,83,-1.125986,0.137652,-0.011666,None,None,None,None,None,None,1
105,105,-1.511348,-0.025152,0.536500,None,None,None,None,None,None,2


In [10]:
sql_output.Decision.value_counts()

2    62
0    50
1    38
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(clf.decision_function(X), columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(columns=['Proba_0', 'Proba_1', 'Proba_2'])
# skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
# skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,-1.209011,-0.594751,0.803762,NaN,NaN,NaN,NaN,NaN,NaN,2
74,74,-0.587840,-0.099202,-0.312959,NaN,NaN,NaN,NaN,NaN,NaN,1
9,9,0.712071,-0.283482,-1.428589,NaN,NaN,NaN,NaN,NaN,NaN,0
88,88,-0.553978,-0.235562,-0.210460,NaN,NaN,NaN,NaN,NaN,NaN,2
25,25,0.620478,-0.244197,-1.376281,NaN,NaN,NaN,NaN,NaN,NaN,0
5,5,1.023440,-1.164134,-0.859305,NaN,NaN,NaN,NaN,NaN,NaN,0
48,48,1.031527,-0.898288,-1.133239,NaN,NaN,NaN,NaN,NaN,NaN,0
117,117,-1.180383,-0.767647,0.948030,NaN,NaN,NaN,NaN,NaN,NaN,2
83,83,-1.125986,0.137652,-0.011666,NaN,NaN,NaN,NaN,NaN,NaN,1
105,105,-1.511348,-0.025152,0.536500,NaN,NaN,NaN,NaN,NaN,NaN,2


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
51,51,-0.561681,-0.448871,0.010552,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.561681,-0.448871,0.010552,None,None,None,None,None,None,2
139,139,-1.022018,-0.533700,0.555717,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.022018,-0.533700,0.555717,None,None,None,None,None,None,2
58,58,-0.695258,0.005459,-0.310201,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.695258,0.005459,-0.310201,None,None,None,None,None,None,1
109,109,-1.111365,-1.044555,1.155920,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.111365,-1.044555,1.155920,None,None,None,None,None,None,2
133,133,-1.027848,0.123116,-0.095267,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.027848,0.123116,-0.095267,None,None,None,None,None,None,1
85,85,-0.531671,-0.691886,0.223557,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.531671,-0.691886,0.223557,None,None,None,None,None,None,2
89,89,-0.757161,0.164887,-0.407725,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.757161,0.164887,-0.407725,None,None,None,None,None,None,1
36,36,1.051588,-0.804471,-1.247117,NaN,NaN,NaN,NaN,NaN,NaN,...,1.051588,-0.804471,-1.247117,None,None,None,None,None,None,0
60,60,-0.794848,0.674202,-0.879354,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.794848,0.674202,-0.879354,None,None,None,None,None,None,1
45,45,0.670550,-0.393396,-1.277154,NaN,NaN,NaN,NaN,NaN,NaN,...,0.670550,-0.393396,-1.277154,None,None,None,None,None,None,0


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
